In [ ]:
# Importing the required libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# Importing the school data -- institutional attributes
# Setting the path of the csv file
data_file = Path("./data/hd2020.csv")
df_schools = pd.read_csv(
    data_file,
    index_col=False
)

In [ ]:
# Importing the completions data by program
# Setting the path of the csv file
data_file = Path("./data/c2020_a.csv")
df_completions = pd.read_csv(data_file,sep=',', index_col=False)
df_completions.describe()

In [ ]:
# Importing the CIP-SOC Crosswalk
# Setting the path of the csv file
data_file = Path("./data/ONET_CIP-SOC_Crosswalk.csv")
# Importing the data file and setting the date column as the index column.
df_CIPlist = pd.read_csv(data_file,sep=',', index_col=False)

In [ ]:
df_completions

In [ ]:
#Slice completions data for just the institution, the program, the award level, and total completions
df_degrees = df_completions.loc[:,["UNITID","CIPCODE","AWLEVEL","CTOTALT"]]
#Aggregate completions by institution, program and award level
df_completions=df_degrees.groupby(["UNITID","CIPCODE","AWLEVEL"]).sum()


In [ ]:
df_completions

In [ ]:
df_completions.reset_index(inplace = True)

In [ ]:
df_completions

In [ ]:
#Review CIPList. It is a crosswalk between programs and occupations.
df_CIPlist

In [ ]:
# Change the column name for CIPCODE so it matches the other dataframes.
df_CIPlist.rename(columns = {'CIP':'CIPCODE'}, inplace = True)

In [ ]:
#Review programs list. It is a crosswalk between occupations and programs.
#df_programslist

In [ ]:
# Change the column name for CIPCODE so it matches the other dataframes.
#df_programslist.rename(columns = {'CIPCode':'CIPCODE'}, inplace = True)

In [ ]:
#Filtered the schools dataframe to get a list of Charlotte CBSA schools
df_NCschools = df_schools.loc[(df_schools["CBSA"] == 16740),["UNITID","INSTNM"]]

In [ ]:
#Reivew the NC Schools list
df_NCschools

In [ ]:
#This is code to concatenate together CIP Codes for the vocational programs from the cost files. It will not be needed.
#df1 = df_programs.loc[:,["UNITID","CIPCODE1"]]
#df2 = df_programs.loc[df_programs["CIPCODE2"] != -2.000,["UNITID","CIPCODE2"]]
#df3 = df_programs.loc[df_programs["CIPCODE3"] != -2.000,["UNITID","CIPCODE3"]]
#df4 = df_programs.loc[df_programs["CIPCODE4"] != -2.000,["UNITID","CIPCODE4"]]
#df5 = df_programs.loc[df_programs["CIPCODE5"] != -2.000,["UNITID","CIPCODE5"]]
#df6 = df_programs.loc[df_programs["CIPCODE6"] != -2.000,["UNITID","CIPCODE6"]]

#df1.rename(columns = {'CIPCODE1':'CIPCODE'}, inplace = True)
#df2.rename(columns = {'CIPCODE2':'CIPCODE'}, inplace = True)
#df3.rename(columns = {'CIPCODE3':'CIPCODE'}, inplace = True)
#df4.rename(columns = {'CIPCODE4':'CIPCODE'}, inplace = True)
#df5.rename(columns = {'CIPCODE5':'CIPCODE'}, inplace = True)
#df6.rename(columns = {'CIPCODE6':'CIPCODE'}, inplace = True)

#frames = [df1,df2,df3,df4,df5,df6]

#df_CIPS = pd.concat(frames)

#df_CIPS

In [ ]:
#Create the NC programs file by merging NC schools with the completions dataframe with an inner join
df_NCprograms = pd.merge(df_NCschools, df_completions, on='UNITID', how ='left')

In [ ]:
df_NCprograms

In [ ]:
#Finally, join NC Programs with our CIP/SOC crosswalk 
df_NCprograms_withSOC = pd.merge(df_NCprograms,df_CIPlist,on='CIPCODE', how ='left')


In [ ]:
df_NCprograms_withSOC

In [ ]:
cols = df_NCprograms_withSOC.columns.tolist()

cols = ['UNITID','INSTNM','CIPCODE','CIP_Title','AWLEVEL','CTOTALT', 'SOC', 'SOC_Title']

df_NCprograms_withSOC = df_NCprograms_withSOC[cols]
#df_NCprograms_withSOC = df_NCprograms_withSOC.drop_duplicates()

In [ ]:
df_NCprograms_withSOC

In [ ]:
AtriumSOCList = [
'29-2034.00',
'29-2032.00',
'31-9092.00',
'29-2010.00',
'29-2011.00',
'29-1141.00',
'31-1131.00',
'29-2061.00',
'29-2040.00',
'29-2055.00',
'29-2099.00'
]

df_AtriumPrograms = df_NCprograms_withSOC[df_NCprograms_withSOC['SOC'].isin(AtriumSOCList)]
df_AtriumPrograms

In [ ]:
df_AtriumSchools = df_AtriumPrograms[['UNITID','INSTNM']]
df_AtriumSchools = df_AtriumSchools.drop_duplicates()
df_AtriumSchools = pd.merge(df_AtriumSchools, df_schools[['UNITID','WEBADDR']], on='UNITID', how ='left')
df_AtriumSchools

In [ ]:
df_NCprograms_withSOC.to_csv('ProgramsbySOC.csv', index=False)

In [ ]:
df_AtriumPrograms.to_csv('AtriumPrograms.csv', index=False)
df_AtriumSchools.to_csv('AtriumSchools.csv', index=False)